In [ ]:
%matplotlib inline
import pandas as pd
from ipywidgets import interact, interactive, VBox, HBox, interactive_output
from IPython.display import display
import ipywidgets as widgets
import os
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
import numpy as np
import holoviews as hv
from holoviews import streams
from holoviews import opts
hv.extension('bokeh')

In [ ]:
x = %system ls 00layers
patientnums = [i.split('-')[0] for i in x]
patientthicks = [i.split('-')[1].replace('.hdf5','') for i in x]
h5files = ['00layers/'+i for i in x]

In [ ]:
numsunique = []
for i in patientnums:
    if i not in numsunique:
        numsunique.append(i)
thickunique = []
for i in patientthicks:
    if i not in thickunique:
        thickunique.append(i)

In [ ]:
h5f = h5py.File(h5files[0],'r')
for i in h5f['070/L'].keys():
    yunique = [int(j) for j in h5f['070/L/'+str(i)+''].keys()]
    yunique.sort(reverse=True)
    xunique = yunique.copy()
    break
h5f.close()

# Widget to cycle through interpolated images

In [ ]:
h5f = h5py.File(h5files[0],'r')
def ploteye(patient='070',eyeside='L',t=0,gridpts=640,tness='FMT_THICKNESS'):
    yfac=gridpts
    xfac=gridpts
    centerrangemin = int(0.2*gridpts)
    centerrangemax = int(0.8*gridpts)
    
    fig, ax = plt.subplots(figsize=(4, 4))
    tstr = str(t)
    if len(h5f.keys()) == 0:
        return('Patient '+patient+
               ',all NaNs (or unable to be interpolated)')
    elif eyeside not in h5f[patient].keys():
        return('Patient '+patient+' eyeside '+eyeside+
               ',all NaNs (or unable to be interpolated)')
    elif tstr not in h5f[patient+'/'+eyeside].keys():
        return('Patient '+patient+' eyeside '+eyeside+' filenum '+tstr+
               ', all NaNs (or unable to be interpolated)')
    elif str(yfac) not in h5f[patient+'/'+eyeside+'/'+tstr].keys():
        return('Patient '+patient+' eyeside '+eyeside+' filenum '+tstr+' resolution '+str(yfac)+
               ', all NaNs (or unable to be interpolated)')
    else:
        tmpdata = h5f[patient+'/'+eyeside+'/'+tstr+'/'+str(yfac)+'/'+str(xfac)+'/dataset'][:]
        im = ax.imshow(tmpdata,
                  aspect=1,
                  origin='lower', cmap='inferno',extent=[0, 24, 0, 24], 
                  vmin=np.amin(tmpdata[centerrangemin:centerrangemax,centerrangemin:centerrangemax]),
                       vmax=np.amax(tmpdata[centerrangemin:centerrangemax,centerrangemin:centerrangemax]))
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        fig.colorbar(im,cax=cax)
        ax.set_title('Date = '+h5f[patient+'/'+eyeside+'/'+tstr+'/'+str(yfac)+'/'+str(xfac)+'/dataset'].attrs['date'])
        plt.tight_layout()

d1 = widgets.Dropdown(options=numsunique,description='Patient')
d2 = widgets.Dropdown(options=['L','R'],description='Eyeside')
d3 = widgets.Dropdown(options=thickunique,description='Layer')
d4 = widgets.Dropdown(options=yunique,description='Grid Pts')
s1 = widgets.IntSlider(min=0,
                       max=len(h5f['070/L'].keys())-1)

def valued1_changed(change):
    global h5f
    h5f.close()
    h5f = h5py.File('00layers/'+change.new+'-'+d3.value+'.hdf5','r')
    if len(h5f.keys()) == 0:
        s1.max = 0
    elif d2.value not in h5f[change.new].keys():
        s1.max = 0
    else:
        s1.max = max(0,len(h5f[change.new+'/'+d2.value].keys())-1)
    s1.value = 0
d1.observe(valued1_changed, 'value')

def valued2_changed(change):
    if len(h5f.keys()) == 0:
        s1.max = 0
    elif change.new not in h5f[d1.value].keys():
        s1.max = 0
    else:
        s1.max = max(0,len(h5f[d1.value+'/'+change.new].keys())-1)
    s1.value = 0
d2.observe(valued2_changed, 'value')

def valued3_changed(change):
    global h5f
    h5f.close()
    h5f = h5py.File('00layers/'+d1.value+'-'+change.new+'.hdf5','r')
    if len(h5f.keys()) == 0:
        s1.max = 0
    elif d2.value not in h5f[d1.value].keys():
        s1.max = 0
    else:
        s1.max = max(0,len(h5f[d1.value+'/'+d2.value].keys())-1)
    s1.value = 0
d3.observe(valued3_changed, 'value')

interact(ploteye, 
         patient=d1, 
         eyeside=d2,
         tness=d3,
         gridpts=d4,
         t=s1);

# Widget to look at differences between images

In [ ]:
h5f = h5py.File(h5files[0],'r')
sortedxfac=[int(i) for i in h5f['070/L/0'].keys()]
sortedxfac.sort(reverse=True)

def ploteyecutoff(base_patient='070',base_eyeside='L',base_t=1,xfac=32,
           patient='070',eyeside='L',t=1,cutval=5.0,tness='FMT_THICKNESS'):

    yfac=xfac
    centerrangemin = int(0.2*xfac)
    centerrangemax = int(0.8*yfac)
    tstr = str(t)
    base_tstr = str(base_t)
    
    if len(h5f.keys()) == 0:
        return('Patient '+patient+
               ',all NaNs (or unable to be interpolated)')
    elif eyeside not in h5f[patient].keys():
        return('Patient '+patient+' eyeside '+eyeside+
               ',all NaNs (or unable to be interpolated)')
    elif tstr not in h5f[patient+'/'+eyeside].keys():
        return('Patient '+patient+' eyeside '+eyeside+' filenum '+tstr+
               ', all NaNs (or unable to be interpolated)')
    elif str(yfac) not in h5f[patient+'/'+eyeside+'/'+tstr].keys():
        return('Patient '+patient+' eyeside '+eyeside+' filenum '+tstr+' resolution '+str(yfac)+
               ', all NaNs (or unable to be interpolated)')
    elif base_eyeside not in h5f[base_patient].keys():
        return('Patient '+base_patient+' eyeside '+base_eyeside+
               ',all NaNs (or unable to be interpolated)')
    elif base_tstr not in h5f[base_patient+'/'+base_eyeside].keys():
        return('Patient '+base_patient+' eyeside '+base_eyeside+' filenum '+base_tstr+
               ', all NaNs (or unable to be interpolated)')
    elif str(yfac) not in h5f[base_patient+'/'+base_eyeside+'/'+base_tstr].keys():
        return('Patient '+base_patient+' eyeside '+base_eyeside+' filenum '+base_tstr+' resolution '+str(yfac)+
               ', all NaNs (or unable to be interpolated)')
    else:
        fig, ax = plt.subplots(1,4,figsize=(14, 8))
        tmpdatabase = h5f[base_patient+'/'+base_eyeside+'/'+str(base_t)+
                         '/'+str(yfac)+'/'+str(xfac)+'/dataset']
        im = ax[0].imshow(tmpdatabase[:],
                     aspect=1,
                     origin='lower', cmap='inferno',extent=[0, 24, 0, 24],
                          vmin=np.amin(tmpdatabase[centerrangemin:centerrangemax,centerrangemin:centerrangemax]),
                          vmax=np.amax(tmpdatabase[centerrangemin:centerrangemax,centerrangemin:centerrangemax]))
        ax[0].set_title('Baseline\nDate = '+tmpdatabase.attrs['date'])
        divider = make_axes_locatable(ax[0])
        cax = divider.append_axes("right", size="5%", pad=0.05)
        fig.colorbar(im,cax=cax)
        tmpdatahere = h5f[patient+'/'+eyeside+'/'+str(t)+
                         '/'+str(yfac)+'/'+str(xfac)+'/dataset']
        im = ax[1].imshow(tmpdatahere[:],
                     aspect=1,
                     origin='lower', cmap='inferno',extent=[0, 24, 0, 24], 
                          vmin=np.amin(tmpdatabase[centerrangemin:centerrangemax,centerrangemin:centerrangemax]),
                          vmax=np.amax(tmpdatabase[centerrangemin:centerrangemax,centerrangemin:centerrangemax]))
        ax[1].set_title('Current\nDate = '+tmpdatahere.attrs['date'])
        divider = make_axes_locatable(ax[1])
        cax = divider.append_axes("right", size="5%", pad=0.05)
        fig.colorbar(im,cax=cax)
        masterdiff = (tmpdatahere[:] - tmpdatabase[:])
        masterdiff[np.isnan(masterdiff)] = 0
        abovecut = masterdiff > (-1*cutval)
        masterdiff[abovecut] = 0
        im = ax[2].imshow(abs(masterdiff), 
                  aspect=1, 
                  origin='lower', cmap='inferno',extent=[0, 24, 0, 24], 
                     vmin=0,vmax=20)
        ax[2].set_title('Lower Vals')
        divider = make_axes_locatable(ax[2])
        cax = divider.append_axes("right", size="5%", pad=0.05)
        fig.colorbar(im,cax=cax)
        masterdiff = (tmpdatahere[:] - tmpdatabase[:])
        masterdiff[np.isnan(masterdiff)] = 0
        belowcut = masterdiff < (cutval)
        masterdiff[belowcut] = 0
        im = ax[3].imshow(masterdiff, 
                  aspect=1, 
                  origin='lower', cmap='inferno',extent=[0, 24, 0, 24], 
                     vmin=0,vmax=20)
        ax[3].set_title('Higher Vals')
        divider = make_axes_locatable(ax[3])
        cax = divider.append_axes("right", size="5%", pad=0.05)
        fig.colorbar(im,cax=cax)
        plt.tight_layout()

d1 = widgets.Dropdown(options=numsunique,description='Base Patient:',style={'description_width': 'initial'})
d2 = widgets.Dropdown(options=['L','R'],description='Base EyeSide:',style={'description_width': 'initial'})
d3 = widgets.Dropdown(options=thickunique)
s1 = widgets.IntSlider(min=0,
                       max=len(h5f['070/L'].keys())-1,description='Base Image #:',style={'description_width': 'initial'})
d12 = widgets.Dropdown(options=numsunique,description='Current Patient:',style={'description_width': 'initial'})
d22 = widgets.Dropdown(options=['L','R'],description='Current EyeSide:',style={'description_width': 'initial'})
s12 = widgets.IntSlider(min=0,
                       max=len(h5f['070/L'].keys())-1,description='Current Image #:',style={'description_width': 'initial'})
c1 = widgets.FloatText(value=0.0,
                       description='Cut-off (um):')
dxfac = widgets.Dropdown(options=sortedxfac,description='Grid Pts')

def valued1_changed(change):
    global h5f
    h5f.close()
    h5f = h5py.File('00layers/'+change.new+'-'+d3.value+'.hdf5','r')
    if len(h5f.keys()) == 0:
        s1.max = 0
    elif d2.value not in h5f[change.new].keys():
        s1.max = 0
    else:
        s1.max = max(0,len(h5f[change.new+'/'+d2.value].keys())-1)
    s12.max = s1.max
    play.max = s12.max
    s1.value = 0
    s12.value = 0
    play.value = 0
    d12.value = d1.value
    d22.value = d2.value
d1.observe(valued1_changed, 'value')

def valued12_changed(change):
    global h5f
    h5f.close()
    h5f = h5py.File('00layers/'+change.new+'-'+d3.value+'.hdf5','r')
    if len(h5f.keys()) == 0:
        s12.max = 0
    elif d22.value not in h5f[change.new].keys():
        s12.max = 0
    else:
        s12.max = max(0,len(h5f[change.new+'/'+d22.value].keys())-1)
    play.max = s12.max
    play.value = 0
    s1.max = s12.max
    s1.value = 0
    s12.value = 0
    d1.value = d12.value
    d2.value = d22.value
d12.observe(valued12_changed, 'value')

def valued2_changed(change):
    if len(h5f.keys()) == 0:
        s1.max = 0
    elif change.new not in h5f[d1.value].keys():
        s1.max = 0
    else:
        s1.max = max(0,len(h5f[d1.value+'/'+change.new].keys())-1)
    s12.max = s1.max
    play.max = s12.max
    play.value = 0
    s1.value = 0
    s12.value = 0
    d22.value = d2.value
d2.observe(valued2_changed, 'value')

def valued22_changed(change):
    if len(h5f.keys()) == 0:
        s12.max = 0
    elif change.new not in h5f[d12.value].keys():
        s12.max = 0
    else:
        s12.max = max(0,len(h5f[d12.value+'/'+change.new].keys())-1)
    play.max = s12.max
    play.value = 0
    s1.max = s12.max
    s1.value = 0
    s12.value = 0
    d2.value = d22.value
d22.observe(valued22_changed, 'value')

def valued3_changed(change):
    global h5f
    h5f.close()
    h5f = h5py.File('00layers/'+d12.value+'-'+change.new+'.hdf5','r')
    if len(h5f.keys()) == 0:
        s12.max = 0
    elif d22.value not in h5f[d12.value].keys():
        s12.max = 0
    else:
        s12.max = max(0,len(h5f[d12.value+'/'+d22.value].keys())-1)
    play.max = s12.max
    play.value = 0
    s1.max = s12.max
    s1.value = 0
    s12.value = 0
    d1.value = d12.value
    d2.value = d22.value
d3.observe(valued3_changed, 'value')

left_box = VBox([d1,d2,s1,d3])
middle_box = VBox([d12,d22,s12])
right_box = VBox([c1,dxfac])
controls = HBox([left_box, middle_box, right_box])

plotwidget = interactive_output(ploteyecutoff, {
         'base_patient':d1, 
         'base_eyeside':d2,
         'base_t':s1,
         'tness':d3,
         'patient':d12, 
         'eyeside':d22, 
         't':s12,
         'cutval':c1,'xfac':dxfac});

play = widgets.Play(value=0,min=0,max=s12.max,step=1,interval=1500,description="Press play",disabled=False)

widgets.jslink((play, 'value'), (s12, 'value'))
aoeu = widgets.VBox([controls,plotwidget,play])

display(aoeu)

# Holoview viz with lineouts

In [ ]:
h5f = h5py.File(h5files[0],'r')
yfac = 32
xfac = 32
sortedxfac=[int(i) for i in h5f['070/L/0'].keys()]
sortedxfac.sort(reverse=True)
sortedxfac.pop(0)
opts.defaults(opts.Text(color='#D3D3D3'))

def eyexy(patient='070',eyeside='L',time=0,xfac=64,tness='FMT_THICKNESS'):

    yfac=xfac
    centerrangemin = int(0.2*xfac)
    centerrangemax = int(0.8*yfac)

    if len(h5f.keys()) == 0:
        return('Patient '+patient+
               ',all NaNs (or unable to be interpolated)')
    elif eyeside not in h5f[patient].keys():
        return('Patient '+patient+' eyeside '+eyeside+
               ',all NaNs (or unable to be interpolated)')
    elif str(time) not in h5f[patient+'/'+eyeside].keys():
        return('Patient '+patient+' eyeside '+eyeside+' filenum '+str(time)+
               ', all NaNs (or unable to be interpolated)')
    elif str(yfac) not in h5f[patient+'/'+eyeside+'/'+str(time)].keys():
        return('Patient '+patient+' eyeside '+eyeside+' filenum '+str(time)+' resolution '+str(yfac)+
               ', all NaNs (or unable to be interpolated)')

    def xycross_hair_info(x, y):
        text = hv.Text(x+0.05, y, '%.3f'% img[x,y], halign='left', valign='bottom')
        return hv.HLine(y) * hv.VLine(x) * text

    if str(time) in h5f[patient+'/'+eyeside].keys():
        f = h5f[patient+'/'+eyeside+'/'+str(time)+
                    '/'+str(yfac)+'/'+str(xfac)+'/dataset'][:]
    else:
        return(str(time)+
               '/'+
               str(xys1.max)+
               ', all NaNs')
    if np.isnan(f).all():
        return('Date = '+
                 h5f[patient+'/'+eyeside+'/'+str(time)+
                     '/'+str(yfac)+'/'+str(xfac)+'/dataset'].attrs['date']+
               ', Image '+
               str(time)+
               '/'+
               str(xys1.max)+
               ', all NaNs')
    else:
        img = hv.Image(f)
        img.opts(clim=(np.amin(f[centerrangemin:centerrangemax,centerrangemin:centerrangemax]),
                                                     np.amax(f[centerrangemin:centerrangemax,centerrangemin:centerrangemax])),
                 title='Date = '+
                 h5f[patient+'/'+eyeside+'/'+str(time)+
                     '/'+str(yfac)+'/'+str(xfac)+'/dataset'].attrs['date']+
                 ', Image '+str(time)+'/'+str(xys1.max))

        pointer = streams.PointerXY(x=0,y=0, source=img)
        pointer_dmapx = hv.DynamicMap(lambda x, y: img.sample(x=np.clip(x,-1,1)), streams=[pointer])
        pointer_dmapx.opts(invert_axes=True,width=(200))
        pointer_dmapy = hv.DynamicMap(lambda x, y: img.sample(y=np.clip(y,-1,1)), streams=[pointer])
        pointer_dmapy.opts(height=(200))
        layout = hv.Layout(
            (pointer_dmapy + 
             hv.Empty() + 
             img * hv.DynamicMap(xycross_hair_info, streams=[pointer]) + 
             pointer_dmapx ).opts(opts.Curve(framewise=True,
                                             #ylim=(0, 120)
                                             #ylim=(np.amin(f[6:26,6:26]), np.amax(f[6:26,6:26]))
                                             ylim = (np.amin(f[centerrangemin:centerrangemax,centerrangemin:centerrangemax]),
                                                     np.amax(f[centerrangemin:centerrangemax,centerrangemin:centerrangemax]))
                                            ))).cols(2)
        return(layout)

xyd1 = widgets.Dropdown(options=numsunique,description='Patient')
xyd2 = widgets.Dropdown(options=['L','R'],description='Eyeside')
xyd3 = widgets.Dropdown(options=thickunique,description='Layer')
xys1 = widgets.IntSlider(min=0,
                       max=len(h5f['070/L'].keys())-1,description='Image #:',
                      continuous_update=False)
dxfac = widgets.Dropdown(options=sortedxfac,description='Grid Pts')

def xyvalued1_changed(change):
    global h5f
    h5f.close()
    h5f = h5py.File('00layers/'+change.new+'-'+xyd3.value+'.hdf5','r')
    if len(h5f.keys()) == 0:
        pass
    elif xyd2.value not in h5f[change.new].keys():
        pass
    else:
        xys1.max = max(0,len(h5f[change.new+'/'+xyd2.value].keys())-1)
    xys1.value = 0
xyd1.observe(xyvalued1_changed, 'value')

def xyvalued2_changed(change):
    if len(h5f.keys()) == 0:
        pass
    elif change.new not in h5f[xyd1.value].keys():
        pass
    else:
        xys1.max = max(0,len(h5f[xyd1.value+'/'+change.new].keys())-1)
    xys1.value = 0
xyd2.observe(xyvalued2_changed, 'value')

def xyvalued3_changed(change):
    global h5f
    h5f.close()
    h5f = h5py.File('00layers/'+xyd1.value+'-'+change.new+'.hdf5','r')
    if len(h5f.keys()) == 0:
        pass
    elif xyd2.value not in h5f[xyd1.value].keys():
        pass
    else:
        xys1.max = max(0,len(h5f[xyd1.value+'/'+xyd2.value].keys())-1)
    xys1.value = 0
xyd3.observe(xyvalued3_changed, 'value')

interact(eyexy, 
         patient=xyd1, 
         eyeside=xyd2, 
         tness=xyd3,
         time=xys1,
         xfac=dxfac);
         #yfac=dyfac);
